In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#-------------------Inputs-------------------------------------------------------------
input_files = ["Shear_and_Moment_4 Paddlers.csv", "Shear_and_Moment_2 Paddlers.csv", "Shear_and_Moment_Display_Stand.csv"]
script_dir = os.path.abspath('') # CSV files will be pulled from this folder.  Use os.path.abspath('') to pull files from the same folder as the script
hf = 25 # [mm]
bw = hf # [mm]
f_c = 10 # [MPa]
alpha_1 = 0.84265
beta_1 = 0.95775
layer1 = 1 # if layer exists, it equals to 1. Otherwise, zero.
layer2 = 0 # if layer exists, it equals to 1. Otherwise, zero.
d_mesh_1 = 12.5 # distance b/w the first layer of reinforcement to the surface [mm]
d_mesh_2 = 0 # distance b/w the second layer of reinforcement to the surface [mm]
epsilon_cu = 0.0035
#-----------Reinforcement Property-------------
phi_s = 0.9 
fy = 4347 # [MPa]
phi_c = 0.65
E_s = 240000 # [MPa]
layer_unit_1 = 46 # one lap for layer 1 [mm^2/m]
layer_unit_2 = 0 # one lap for layer 2
#--------------------------------------------------------------------------------------




In [2]:
# Positive Bending Moment
def print_and_write(a, file):
    print(a)
    file.write(a + "\n")

f = open("Positive_Bending_Moment_Info.txt", mode = "w")
f.write("Positive Bending Moment:\n\n")

# Compute for all three files, display the worst 'value'
for file in input_files:
    
    file_path = os.path.join(script_dir, file)
    info = pd.read_csv(file_path, header=None)

    value = float(info[1][3])
    width = float(info[3][3])
    depth = float(info[4][3])



    bf = width/2
    #b_L = bf-bw
    b_L = 6*hf

    # Step 1: Assume Compression is limited to flange and compute 'a'
    As = (layer1*layer_unit_1 + layer2*layer_unit_2)*(width/1000)/2 # [mm^2]
    a = (phi_s*As*fy)/(phi_c*alpha_1*f_c*bf) #[mm]

    #initialize
    d_bottom = 0

    if layer2 == 0:
        d_bottom = hf - d_mesh_1       
    else:
        d_bottom = hf - (d_mesh_1+d_mesh_2)/2


    if a<=hf:
        Mr = phi_s*As*fy*(d_bottom-a/2)/1000 # Unit change from Nmm to Nm
    else:
        # Flange Component:
        M_flange = phi_c * alpha_1 * f_c * (bf - bw) * hf * (d_bottom - hf/2)

        # Web Component:
        Asw = (phi_c * alpha_1 * f_c * a * bw) / (phi_s * fy)
        a = (phi_s * Asw * fy) / (phi_c * alpha_1 * f_c * bw)
        M_web = phi_s * Asw * fy (d_bottom - a/2)

        # Total Moment:
        Mr = (M_flange + M_web) / 1000 # Unit change from Nmm to Nm


    c = a/beta_1


    print_and_write('In file: ' + file, f)

    if c/d_bottom > (epsilon_cu/E_s)/(fy+epsilon_cu/E_s):
        print_and_write('Over-reinforcement failure (desirable)', f)

    else:
        print_and_write('Under-reinforcement failure (undesirable)', f)
        
    print_and_write('The negative moment that can be resisted by the beam is ' + str(Mr*2) +'Nm.', f)
    print_and_write('The applied moment is ' + str(value) + ' Nm.\n', f)

f.close()


TypeError: 'int' object is not callable

In [ ]:
# Negative Bending Moment
def print_and_write(a, file):
    print(a)
    file.write(a + "\n")

g = open("Negative_Bending_Moment_Info.txt", mode = "w")
g.write("Negative Bending Moment:\n\n")

# Compute for all three files, display the worst 'value'
for file in input_files:

    file_path = os.path.join(script_dir, file)
    info = pd.read_csv(file_path, header=None)

    value = float(info[1][4])
    width = float(info[3][4])
    depth = float(info[4][4])

    

    d_top = depth - d_bottom

    # Web component in flange
    Asw = (layer1)*layer_unit_1*bw/1000
    a = ((phi_s*As*fy)/(phi_c*alpha_1*f_c) - bw*(depth-hf))/(width/2)


    print_and_write('In file: ' + file, g)

    if a>0 and a<hf:
        # Flange Component
        Asf = phi_c*alpha_1*f_c*(a*width/2)/(phi_s*fy)
        M_flange = phi_c*alpha_1*f_c*(a*width/2)*(d_mesh_1 - a/2)
        
        # Web component
        Asw = As - Asf
        M_web = phi_s*Asw*fy*(d_top-(depth-hf)/2)
        
        M_total = (M_flange + M_web)/1000
        print_and_write('Case 1: Neutral axis is in the bottom hull', g)
        print_and_write('The positive moment that can be resisted by the beam is ' + str(M_total*2) + 'Nm.', g)
        
    else:
        a = (phi_s*As*fy)/(phi_c*alpha_1*f_c*bw)
        
        if a>0 and a<(depth-hf):
            M_total = phi_s*As*fy*(d_top-a/2)/1000
            print_and_write('Case 2: Neutral axis is in the wall', g)
            print_and_write('The positive moment that can be resisted by the beam is ' + str(M_total*2) + 'Nm.', g)
        else:
            print_and_write('a = ' + a, g)
            print_and_write('See code notes to fix it.', g)

    print_and_write('The applied moment is ' + str(value) + ' Nm.\n', g)